In [1]:
# coding: utf-8

import networkx as nx
import pandas as pd
import fiona
import geopandas as gpd
import numpy as np
from operator import attrgetter

from vresutils.costdata import annuity
from vresutils.shapes import haversine


In [2]:
# rule build_topology:
#     input:
#         supply_regions='data/supply_regions/supply_regions.shp',
#         centroids='data/supply_regions/centroids.shp',
#         num_lines='data/num_lines.csv'
#     output:
#         buses='resources/buses.csv',
#         lines='resources/lines.csv'
#     threads: 1
#     script: "scripts/build_topology.py"

In [3]:
import yaml
with open("../config.yaml", 'r') as stream:
    try:
        snakemake_config=yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
snakemake_input_supply_regions='../data/supply_regions/supply_regions.shp'
snakemake_input_supply_regions='../data/supply_regions_new/RSA_single.shp'
snakemake_input_centroids='../data/supply_regions/centroids.shp'
snakemake_input_centroids='../data/supply_regions_new/centroid_RSA_single.shp'
snakemake_input_num_lines='../data/num_lines.csv'


In [8]:
## Read in regions and calculate population per region
regions = gpd.read_file(snakemake_input_supply_regions)[['name', 'geometry']]
centroids = gpd.read_file(snakemake_input_centroids).set_index('name')['geometry']



In [ ]:
def edges_between_touching_regions(regions):
    G = nx.Graph()
    G.add_nodes_from(regions.index)
    for r in regions.itertuples():
        neighs = regions.index[regions.touches(r.geometry)]
        G.add_edges_from((r.Index, r2_name) for r2_name in neighs.values)
    return G.edges()

In [11]:

# Slighly Moved centroids of NAMAQUALAND and PRETORIA manually so that they are within the shapes


# touching regions are connected by lines, we use nx to take care of all the double countings efficiently

def edges_between_touching_regions(regions):
    G = nx.Graph()
    G.add_nodes_from(regions.index)
    for r in regions.itertuples():
        neighs = regions.index[regions.touches(r.geometry)]
        G.add_edges_from((r.Index, r2_name) for r2_name in neighs.values)
    return G.edges()

regions = regions.set_index('name')

line_config = snakemake_config['lines']
v_nom = line_config['v_nom']

buses = (regions
            .assign(
                x=centroids.map(attrgetter('x')),
                y=centroids.map(attrgetter('y')),
                v_nom=v_nom
            )
            .drop('geometry', axis=1))

# Lines from touching regions
def asarray(x): return np.asarray(list(map(np.asarray, x)))
lines = pd.DataFrame(list(edges_between_touching_regions(regions)), columns=['bus0', 'bus1'])
lines['length'] = haversine(asarray(lines.bus0.map(centroids)),
                            asarray(lines.bus1.map(centroids))) * line_config['length_factor']

num_lines = pd.read_csv(snakemake_input_num_lines, index_col=0).set_index(['bus0', 'bus1'])
num_parallel = sum(num_lines['num_parallel_{}'.format(int(v))] * (v/v_nom)**2
                    for v in (275, 400, 765))

lines = (lines
            .join(num_parallel.rename('num_parallel'), on=['bus0', 'bus1'])
            .join(num_parallel.rename("num_parallel_i"), on=['bus1', 'bus0']))

lines['num_parallel'] = line_config['s_nom_factor'] * lines['num_parallel'].fillna(lines.pop('num_parallel_i'))




IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [15]:
len(buses)

1

In [18]:
lines=pd.DataFrame(index=[],columns=['name','bus0','bus1','length','num_parallel'])


In [19]:
lines

,name,bus0,bus1,length,num_parallel


In [ ]:
buses, lines = build_topology()

#buses.to_csv(snakemake.output.buses)
#lines.to_csv(snakemake.output.lines)

In [ ]:
regions

NameError: name 'regions' is not defined

In [ ]:
"D:\GitHub\pypsa-za\data\supply_regions_new\zaf_admbnda_adm0_sadb_ocha_20201109.shp"